In [2]:
import copy
import numpy as np
import nltk
import pdb ## in case of tracing back
#pdb.set_trace()
nltk.download('gutenberg')
from nltk.corpus import gutenberg
print(gutenberg.fileids())
import matplotlib.pyplot as plt
from copy import deepcopy
##可变对象（比如字典或者列表），'=' 是进行传址赋值，会修改函数指向的地址内的内容，所以这里需要运用copy.deepcopy，防止原始内容被修改
##不可变数据（3 个）：Number（数字）、String（字符串）、Tuple（元组; 可变数据（3 个）：List（列表）、Dictionary（字典）、Set（集合）。
textname_list=copy.deepcopy(gutenberg.fileids())
print(f'the orignal length of names_list is: {len(textname_list)}')
nltk.download('punkt')
import time
start_time=time.time()


[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']
the orignal length of names_list is: 18
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
##analysis the orignal data, delete the books with words which is less than 100*200
import string
from numpy import *
import pandas as pd
from pandas import DataFrame
data_list=[]

#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)

punctuation=string.punctuation

for name in textname_list:
  text0=gutenberg.raw(name)
  tokenized_sents=nltk.sent_tokenize(text0)
  sents_lenlist=[]
  count=0
  line_num=0
  word_num=0
  for line in tokenized_sents:
    count_sents_words=0
    line_words=nltk.word_tokenize(line)
    for w in line_words:
      if not w in punctuation:
        count_sents_words+=1
        word_num+=1
    sents_lenlist.append(count_sents_words)
    if count_sents_words>=100:
      count+=1
  if word_num<200*100:
    textname_list.remove(name)
  data_list.append([name,len(sents_lenlist),round(mean(sents_lenlist),1),count,word_num/100])
  print(f'sents_lenlist is {sents_lenlist}')
'''
  plt.figure()
  plt.plot(range(1,len(sents_lenlist)+1),sents_lenlist)
  plt.title(name)
'''

data_df=pd.DataFrame(data_list)
data_df.columns=['book ','total_sents ','mean_token ','ideal_sents ','total_token/100']     
print(data_df)
print(f'the number of left books is is {len(textname_list)}')
print(textname_list)

sents_lenlist is [49, 33, 41, 27, 9, 67, 41, 20, 22, 10, 19, 27, 24, 10, 45, 13, 55, 42, 64, 68, 12, 10, 73, 62, 31, 7, 5, 32, 29, 4, 27, 114, 45, 13, 13, 46, 18, 17, 22, 13, 10, 5, 8, 8, 10, 3, 31, 12, 8, 10, 24, 11, 5, 6, 16, 10, 42, 45, 31, 16, 12, 35, 15, 34, 31, 26, 12, 27, 28, 11, 5, 19, 11, 8, 3, 4, 6, 2, 14, 13, 9, 11, 32, 5, 4, 2, 3, 6, 18, 37, 14, 3, 25, 17, 12, 4, 15, 4, 2, 4, 20, 10, 69, 20, 17, 10, 32, 21, 24, 12, 28, 10, 16, 79, 15, 27, 39, 7, 6, 20, 8, 17, 8, 17, 3, 58, 2, 7, 24, 16, 53, 30, 14, 4, 23, 9, 62, 4, 5, 14, 35, 7, 25, 23, 29, 11, 24, 17, 21, 24, 9, 3, 15, 67, 21, 26, 21, 7, 14, 31, 25, 15, 30, 52, 60, 53, 11, 53, 37, 39, 51, 11, 60, 53, 7, 24, 9, 37, 53, 65, 32, 63, 51, 14, 7, 41, 36, 21, 34, 14, 22, 29, 31, 20, 18, 9, 6, 19, 7, 47, 49, 78, 47, 92, 20, 8, 25, 36, 19, 42, 15, 54, 41, 32, 12, 52, 35, 25, 56, 58, 54, 14, 23, 31, 18, 39, 8, 33, 16, 13, 46, 25, 20, 102, 54, 17, 65, 43, 47, 68, 19, 8, 27, 10, 36, 19, 47, 82, 4, 22, 10, 19, 75, 30, 22, 70, 76, 11, 5

In [5]:
## cleaning books: which can not find its author
import re
import string
# don't know the author of the forth book: new book4 --[The King James Bible]-->remove
book_author={}
authorlist=[]

for name in textname_list:
    tokenized_sents_eachbook=nltk.sent_tokenize(gutenberg.raw(name))
    for line in tokenized_sents_eachbook:
      ## why not author = re.match(r'[.* by (.*) [0-9]+]',line), the left '[' is not highlighted as the right/close one
      matchobj=re.match(r'.* by (.*) [0-9]*]',line)
      if matchobj:
        author = (matchobj).group(1)
        break
      else:
        author=None
    book_author[name]=author

for k,v in book_author.items():
  if v==None:
    textname_list.remove(k)
  else:
    authorlist.append(k)

print(len(authorlist))
print(f'the number of left books is {len(textname_list)}')
print(textname_list)

print(data_df.loc[data_df['book '].isin(textname_list)])
print(type(data_df))

13
the number of left books is 13
['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bryant-stories.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'whitman-leaves.txt']
                      book   total_sents   mean_token   ideal_sents   \
0           austen-emma.txt          7493         22.5            91   
1     austen-persuasion.txt          3654         23.4            36   
2          austen-sense.txt          4833         25.8            52   
6       chesterton-ball.txt          4624         18.7             3   
7      chesterton-brown.txt          3712         20.4             1   
8   chesterton-thursday.txt          3588         17.1             0   
10   melville-moby_dick.txt          9852         22.4            69   
11      milton-paradise.txt          1835         43.8           152   
12   shakespe

In [ ]:
#randomly pick 5 book with different author and label it

import random
random.seed(0)
random_num_list=[]
author_list=[]
author_list_idx={}
num_book=0
while num_book<5:
  # randint(a,b) means the value between a and b(including a and b), so len(textname_list) should minus 1, in case of out of index range
  random_num = random.randint(0,len(textname_list)-1)
  random_author = book_author[textname_list[random_num]]
  if (random_num not in random_num_list and random_author not in author_list):
    random_num_list.append(random_num)
    author_list.append(random_author)
    num_book+=1

for i in range(len(author_list)):
  author_list_idx[author_list[i]]=i+1

print(author_list_idx)
print(list(zip(random_num_list,author_list)))

{'G. K. Chesterton': 1, 'Walt Whitman': 2, 'Jane Austen': 3, 'Lewis Carroll': 4, 'Herman Melville': 5}
[(6, 'G. K. Chesterton'), (12, 'Walt Whitman'), (0, 'Jane Austen'), (4, 'Lewis Carroll'), (8, 'Herman Melville')]


In [ ]:
## extract 200 partition and each partition has 100 tokens
import numpy
from sklearn.model_selection import cross_val_score
sents_author=[]
for i, author in zip(random_num_list,author_list):
  book_token_list=[] 
  text=gutenberg.raw(textname_list[i])
  tokenized_sents=nltk.sent_tokenize(text)
  tokenized_words=nltk.word_tokenize(text)
  for w in tokenized_words:
    if w not in punctuation:
      book_token_list.append(w)
  for i in range(0,200):
    sent=''
    for t in book_token_list[((i+1)*100-100):(i+1)*100]:
      sent=sent+' '+t
    sents_author.append((sent,author))

random.seed(2)
random.shuffle(sents_author)
print(numpy.array(sents_author[0:5]))
print(len(sents_author))
#print(sents_author)
#sents_author: [(' The Wisdom of Father Brown by G. K. Chesterton 1914 I The Absence of Mr Glass THE consulting-rooms of Dr Orion Hood the eminent criminologist and specialist in certain moral disorders lay along the sea-front at Scarborough in a series of very large and well-lighted french windows which showed the North Sea like one endless outer wall of blue-green marble In such a place the sea had something of the monotony of a blue-green dado for the chambers themselves were ruled throughout by a terrible tidiness not unlike the terrible tidiness of the sea It must not be supposed that', 'G. K. Chesterton'), ..]
    

[[" as they tug at her teats The brood of the turkey-hen and she with her half-spread wings I see in them and myself the same old law The press of my foot to the earth springs a hundred affections They scorn the best I can do to relate them I am enamour 'd of growing out-doors Of men that live among cattle or taste of the ocean or woods Of the builders and steerers of ships and the wielders of axes and mauls and the drivers of horses I can eat and sleep with them week in and week out"
  'Walt Whitman']
 [" songs No special strains to sing none for itself But from the whole resulting rising at last and floating A round full-orb 'd eidolon. For Him I Sing For him I sing I raise the present on the past As some perennial tree out of its roots the present on the past With time and space I him dilate and fuse the immortal laws To make himself by them the law unto himself. When I Read the Book When I read the book the biography famous And is this then said I what the author calls a man 's lif

In [ ]:
#May be need to run twice to download some lib
!pip install pandas==0.24.2
!pip install scikit-learn==0.21.1
!pip install pandas_ml

#define the function to caculate performance index: confusion_matrix, precision, recall, F1...
from pandas_ml import ConfusionMatrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_predict

def model_performance(model_name, Y_real, Y_pred):
  print(f'the performance of {model_name}:')
  print(f'confusion_matrix is:')
  print(ConfusionMatrix(Y_real,Y_pred))
  print(f"precision is: {precision_score(Y_real,Y_pred,average='micro')}")
  print(f"recall is: {recall_score(Y_real,Y_pred,average='micro')}")
  print(f"F1_score is: {f1_score(Y_real,Y_pred,average='micro')}")


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [ ]:
#BOW based on the orignal text without cleaning
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
np.set_printoptions(threshold=np.inf)

corpus,authors=zip(*sents_author)
Y_BOW_Orignal=[]
for i in range(len(authors)):
  Y_BOW_Orignal.append(author_list_idx[authors[i]])
vectorizer = CountVectorizer()
X_Orignal = vectorizer.fit_transform(corpus)
X_BOW_Orignal=X_Orignal.toarray()
word_BOW = vectorizer.get_feature_names() 
X_train_BOW_Orignal,X_test_BOW_Orignal=X_BOW_Orignal[0:round(len(X_BOW_Orignal)*0.8)],X_BOW_Orignal[round(len(X_BOW_Orignal)*0.8):len(X_BOW_Orignal)]
Y_train_BOW_Orignal,Y_test_BOW_Orignal=Y_BOW_Orignal[0:round(len(X_BOW_Orignal)*0.8)],Y_BOW_Orignal[round(len(X_BOW_Orignal)*0.8):len(X_BOW_Orignal)]

'''
print(len((X_train_BOW_Orignal[1])))
print(X_train_BOW_Orignal[1])
print(Y_train_BOW_Orignal[1])
'''
##built BOW_Oringal_KNN model
import numpy as np
from sklearn import neighbors
BOW_Orignal_KNN=neighbors.KNeighborsClassifier()
BOW_Orignal_KNN_accuracy=cross_val_score(BOW_Orignal_KNN, X_BOW_Orignal,Y_BOW_Orignal,cv=10)
Y_pred=cross_val_predict(BOW_Orignal_KNN,X_BOW_Orignal,Y_BOW_Orignal,cv=10)
model_performance('BOW_Orignal_KNN', Y_BOW_Orignal, Y_pred)

##built BOW_SVC model
from sklearn.svm import SVC
BOW_Orignal_SVC=SVC(kernel='linear', probability=True)
#pdb.set_trace()
BOW_Orignal_SVC_accuracy=cross_val_score(BOW_Orignal_SVC,X_BOW_Orignal,Y_BOW_Orignal,cv=10)
Y_pred=cross_val_predict(BOW_Orignal_SVC,X_BOW_Orignal,Y_BOW_Orignal,cv=10)
model_performance('BOW_Orignal_SVC', Y_BOW_Orignal, Y_pred)

##built BOW_DC model
from sklearn.tree import DecisionTreeClassifier
BOW_Orignal_DC=DecisionTreeClassifier(criterion='entropy')
BOW_Orignal_DC_accuracy=cross_val_score(BOW_Orignal_DC,X_BOW_Orignal,Y_BOW_Orignal,cv=10)
Y_pred=cross_val_predict(BOW_Orignal_DC,X_BOW_Orignal,Y_BOW_Orignal,cv=10)
model_performance('BOW_Orignal_DC', Y_BOW_Orignal, Y_pred)

print(f'BOW_Orignal_KNN_accuracy is {BOW_Orignal_KNN_accuracy}')
print(f'BOW_Orignal_SVC_accuracy is {BOW_Orignal_SVC_accuracy}')
print(f'BOW_Orignal_DC_accuracy is {BOW_Orignal_DC_accuracy}')

print('BOW_Orignal_KNN_accuracy: %0.2f (+/- %0.2f)' % (BOW_Orignal_KNN_accuracy.mean(), BOW_Orignal_KNN_accuracy.std()*2))
print('BOW_Orignal_SVC_accuracy: %0.2f (+/- %0.2f)' % (BOW_Orignal_SVC_accuracy.mean(), BOW_Orignal_SVC_accuracy.std()*2))
print('BOW_Orignal_DC_accuracy: %0.2f (+/- %0.2f)' % (BOW_Orignal_DC_accuracy.mean(),BOW_Orignal_DC_accuracy.std()*2))

##****word_BOW  is the key word lib


the performance of BOW_Orignal_KNN:
confusion_matrix is:
Predicted    1    2    3    4    5  __all__
Actual                                     
1          152    3    8    4   33      200
2           41  120    8    1   30      200
3           16    0  178    3    3      200
4            7    0   11  175    7      200
5           48    6   11    2  133      200
__all__    264  129  216  185  206     1000
precision is: 0.758
recall is: 0.758
F1_score is: 0.7579999999999999
the performance of BOW_Orignal_SVC:
confusion_matrix is:
Predicted    1    2    3    4    5  __all__
Actual                                     
1          182    5    1    2   10      200
2            4  190    2    0    4      200
3            3    2  193    0    2      200
4            3    2    1  193    1      200
5            6    9    3    3  179      200
__all__    198  208  200  198  196     1000
precision is: 0.937
recall is: 0.937
F1_score is: 0.937
the performance of BOW_Orignal_DC:
confusion_matrix is:
P

In [ ]:
#TFIDF based on the orignal text
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
#a = [('What , know you not ( Being Mechanicall ) you ought not walke Vpon a labouring day , without the signe Of your Profession ?', 'William Shakespeare'),('aaaaaaaaaaaaaaaaaa','William Shakespeare')]
corpus,authors=zip(*sents_author)
Y_TFIDF_Orignal=[]
for i in range(len(authors)):
  Y_TFIDF_Orignal.append(author_list_idx[authors[i]])
print(f' orignal corpus is:')
print(np.array(list(corpus)[0:5]))
print(f'Y_set_TFIDF_Orignal[0:5] is {Y_TFIDF_Orignal[0:5]}')
#pdb.set_trace()
vectorizer = CountVectorizer()
X_Orignal = vectorizer.fit_transform(corpus)
word = vectorizer.get_feature_names() 
print(f'word_len is {len(word)}')
transformer = TfidfTransformer()
X_TFIDF_Orignal = transformer.fit_transform(X_Orignal)
X_set_TFIDF_Orignal=X_TFIDF_Orignal.toarray()
print('X_set_TFIDF_Orignal[0:5] is:')
print(X_set_TFIDF_Orignal[0:5])
print(f'len(X_set_TFIDF_Orignal is {len(X_set_TFIDF_Orignal)}')
print(f'len(Y_TFIDF_Orignal) is {len(Y_TFIDF_Orignal)}')
X_train_TFIDF_Orignal, X_test_TFIDF_Orignal=X_set_TFIDF_Orignal[0:round(len(X_set_TFIDF_Orignal)*0.8)],X_set_TFIDF_Orignal[round(len(X_set_TFIDF_Orignal)*0.8):len(X_set_TFIDF_Orignal)]
Y_train_TFIDF_Orignal, Y_test_TFIDF_Orignal=Y_TFIDF_Orignal[0:round(len(X_set_TFIDF_Orignal)*0.8)],Y_TFIDF_Orignal[round(len(X_set_TFIDF_Orignal)*0.8):len(X_set_TFIDF_Orignal)]

##****word is the key word lib


 orignal corpus is:
[" as they tug at her teats The brood of the turkey-hen and she with her half-spread wings I see in them and myself the same old law The press of my foot to the earth springs a hundred affections They scorn the best I can do to relate them I am enamour 'd of growing out-doors Of men that live among cattle or taste of the ocean or woods Of the builders and steerers of ships and the wielders of axes and mauls and the drivers of horses I can eat and sleep with them week in and week out"
 " songs No special strains to sing none for itself But from the whole resulting rising at last and floating A round full-orb 'd eidolon. For Him I Sing For him I sing I raise the present on the past As some perennial tree out of its roots the present on the past With time and space I him dilate and fuse the immortal laws To make himself by them the law unto himself. When I Read the Book When I read the book the biography famous And is this then said I what the author calls a man 's lif

In [ ]:
##built TFIDF_Oringal_KNN model
import numpy as np
from sklearn import neighbors
TFIDF_Orignal_KNN=neighbors.KNeighborsClassifier()

Y_pred=cross_val_predict(TFIDF_Orignal_KNN,X_set_TFIDF_Orignal,Y_TFIDF_Orignal,cv=10)
model_performance('TFIDF_Orignal_KNN', Y_TFIDF_Orignal, Y_pred)
TFIDF_Orignal_KNN_accuracy=cross_val_score(TFIDF_Orignal_KNN,X_set_TFIDF_Orignal,Y_TFIDF_Orignal,cv=10)

##built TFIDF_SVC model
from sklearn.svm import SVC
TFIDF_Orignal_SVC=SVC(kernel='linear', probability=True)
Y_pred=cross_val_predict(TFIDF_Orignal_SVC,X_set_TFIDF_Orignal,Y_TFIDF_Orignal,cv=10)
model_performance('TFIDF_Orignal_SVC', Y_TFIDF_Orignal, Y_pred)
TFIDF_Orignal_SVC_accuracy=cross_val_score(TFIDF_Orignal_SVC,X_set_TFIDF_Orignal,Y_TFIDF_Orignal,cv=10)

##built TFIDF_DC model
from sklearn.tree import DecisionTreeClassifier
TFIDF_Orignal_DC=DecisionTreeClassifier(criterion='entropy')

Y_pred=cross_val_predict(TFIDF_Orignal_DC,X_set_TFIDF_Orignal,Y_TFIDF_Orignal,cv=10)
model_performance('TFIDF_Orignal_DC', Y_TFIDF_Orignal, Y_pred)

TFIDF_Orignal_DC_accuracy=cross_val_score(TFIDF_Orignal_DC,X_set_TFIDF_Orignal,Y_TFIDF_Orignal,cv=10)

print(f'TFIDF_Orignal_KNN_accuracy is {TFIDF_Orignal_KNN_accuracy}')
print(f'TFIDF_Orignal_SVC_accuracy is {TFIDF_Orignal_SVC_accuracy}')
print(f'TFIDF_Orignal_DC_accuracy is {TFIDF_Orignal_DC_accuracy}')

print('TFIDF_Orignal_KNN_accuracy:%0.2f (+/- %0.2f)' % (TFIDF_Orignal_KNN_accuracy.mean(), TFIDF_Orignal_KNN_accuracy.std()*2))
print('TFIDF_Orignal_SVC_accuracy: %0.2f (+/- %0.2f)' % (TFIDF_Orignal_SVC_accuracy.mean(), TFIDF_Orignal_SVC_accuracy.std()*2))
print('TFIDF_Orignal_DC_accuracy: %0.2f (+/- %0.2f)' % (TFIDF_Orignal_DC_accuracy.mean(), TFIDF_Orignal_DC_accuracy.std()*2))

the performance of TFIDF_Orignal_KNN:
confusion_matrix is:
Predicted    1    2    3    4    5  __all__
Actual                                     
1          170    4   14   11    1      200
2            1  184    9    5    1      200
3            1    2  192    5    0      200
4            0    0    2  198    0      200
5           11   18   21   20  130      200
__all__    183  208  238  239  132     1000
precision is: 0.874
recall is: 0.874
F1_score is: 0.874
the performance of TFIDF_Orignal_SVC:
confusion_matrix is:
Predicted    1    2    3    4    5  __all__
Actual                                     
1          194    3    0    0    3      200
2            1  199    0    0    0      200
3            1    1  195    0    3      200
4            0    2    0  198    0      200
5            1    6    1    0  192      200
__all__    197  211  196  198  198     1000
precision is: 0.978
recall is: 0.978
F1_score is: 0.978
the performance of TFIDF_Orignal_DC:
confusion_matrix is:
Predicte

In [ ]:
pip install spacy

In [ ]:
##cleaning data: lower-->drop punctuation-->identify special entities/drop stopwords-->lemmatization
##count words
import spacy
import spacy
import nltk
import math
import string
import nltk.stem
from nltk.corpus import stopwords
from collections import Counter
nltk.download('stopwords')
punctuation = [char for char in string.punctuation]
s=nltk.stem.SnowballStemmer('english')
nlp=spacy.load('en_core_web_sm')

def clean_text(sents_author):
  result=[]
  word_set=[]
  for corpus_author in sents_author:
    text, label = corpus_author[0], corpus_author[1]
    lower_text = text.lower()
    tokens=nltk.word_tokenize(lower_text)
    drop_punctuation=[w for w in tokens if w not in punctuation]
    doc=nlp(lower_text)
    ent_label =[(ent.text, ent.label_) for ent in doc.ents]
    entities = []
    ## in case of warning because there is not entities in one text
    if ent_label:
      entities, label_=zip(*ent_label) 
    #ent_stopwords=[w for w in drop_punctuation if w in entities]#only keep the entities
    ent_stopwords=[w for w in drop_punctuation if w in entities or not w in stopwords.words('english')]#keep entities but drop stopwords
    #ent_stopwords=[w for w in drop_punctuation]
    clean_text = []
    for i in ent_stopwords:
      clean_text.append(s.stem(i))
    word_set.extend(clean_text)
    count = dict(Counter(clean_text))
    result.append((count,label))
  return result, word_set


wordcount_author,word_set=clean_text(sents_author)
word_set.sort()
word_set=set(word_set)
##****word_set is the key word lib
'''
# data clean result:
#[({'know': 1, 'mechanical': 1}, 'William Shakespeare'), ({'trade': 1, 'sir': 1, 'hope': 1, 'may': 1, 'vse': 1}, 'William Shakespeare')]

for i in range(len(wordcount_author)):
  print(wordcount_author[i])

print(word_set)
print(len(word_set))
'''

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


"\n# data clean result:\n#[({'know': 1, 'mechanical': 1}, 'William Shakespeare'), ({'trade': 1, 'sir': 1, 'hope': 1, 'may': 1, 'vse': 1}, 'William Shakespeare')]\n\nfor i in range(len(wordcount_author)):\n  print(wordcount_author[i])\n\nprint(word_set)\nprint(len(word_set))\n"

In [ ]:
#padding features:
def padding_features(wordcount_author):
  features=[]
  for pair in wordcount_author:
    item,author = pair[0], pair[1]
    for word in word_set:
      if not word in item:
        item[word]=0
      else:
        continue
    item_sorted={k:item[k] for k in sorted(item.keys(),reverse=True)}
    features.append((item_sorted,author))
  return features

BOW_features_author=padding_features(wordcount_author)

X_set,Y_set=zip(*BOW_features_author)

X_set_value=[list(item.values()) for item in X_set]
Y_set_value=[author_list_idx[author] for author in Y_set]

X_train, X_test=X_set_value[0:round(len(X_set_value)*0.8)],X_set_value[round(len(X_set_value)*0.8):len(X_set)]
Y_train, Y_test=Y_set_value[0:round(len(X_set_value)*0.8)],Y_set_value[round(len(X_set_value)*0.8):len(X_set)]
'''
for i in range(len(BOW_BOW_features_author)):
  print(BOW_BOW_BOW_features_author[i])

print(f'X_set is {X_set}')
print(f'Y_set is {Y_set}')

print(f'X_set_value is {X_set_value}')
print(f'Y_set_value is {Y_set_value}')
print(Y_train)
'''

"\nfor i in range(len(BOW_BOW_features_author)):\n  print(BOW_BOW_BOW_features_author[i])\n\nprint(f'X_set is {X_set}')\nprint(f'Y_set is {Y_set}')\n\nprint(f'X_set_value is {X_set_value}')\nprint(f'Y_set_value is {Y_set_value}')\nprint(Y_train)\n"

In [ ]:
##built BOW_KNN model
import numpy as np
from sklearn import neighbors
BOW_KNN=neighbors.KNeighborsClassifier()
Y_pred=cross_val_predict(BOW_KNN,X_set_value,Y_set_value,cv=10)
model_performance('BOW_KNN', Y_set_value, Y_pred)
BOW_KNN_accuracy=cross_val_score(BOW_KNN, X_set_value,Y_set_value,cv=10)

##built BOW_SVC model
from sklearn.svm import SVC
BOW_SVC=SVC(kernel='linear', probability=True)
Y_pred=cross_val_predict(BOW_SVC,X_set_value,Y_set_value,cv=10)
model_performance('BOW_SVC', Y_set_value, Y_pred)
BOW_SVC_accuracy=cross_val_score(BOW_SVC,X_set_value,Y_set_value,cv=10)

##built BOW_DC model
from sklearn.tree import DecisionTreeClassifier
BOW_DC=DecisionTreeClassifier(criterion='entropy')
Y_pred=cross_val_predict(BOW_DC,X_set_value,Y_set_value,cv=10)
model_performance('BOW_DC', Y_set_value, Y_pred)
BOW_DC_accuracy=cross_val_score(BOW_DC, X_set_value,Y_set_value, cv=10)

print(f'BOW_KNN_accuracy is {BOW_KNN_accuracy}')
print(f'BOW_SVC_accuracy is {BOW_SVC_accuracy}')
print(f'BOW_DC_accuracy is {BOW_DC_accuracy}')

print('BOW_KNN_accuracy: %0.2f (+/- %0.2f)' % (BOW_KNN_accuracy.mean(), BOW_KNN_accuracy.std()*2))
print('BOW_SVC_accuracy: %0.2f (+/- %0.2f)' % (BOW_SVC_accuracy.mean(), BOW_SVC_accuracy.std()*2))
print('BOW_DC_accuracy: %0.2f (+/- %0.2f)' % (BOW_DC_accuracy.mean(), BOW_DC_accuracy.std()*2))


the performance of BOW_KNN:
confusion_matrix is:
Predicted    1    2    3    4    5  __all__
Actual                                     
1          146    2   10    2   40      200
2           14  158   10    0   18      200
3            5    0  185    4    6      200
4            7    0   14  175    4      200
5           32    7   12    5  144      200
__all__    204  167  231  186  212     1000
precision is: 0.808
recall is: 0.808
F1_score is: 0.808
the performance of BOW_SVC:
confusion_matrix is:
Predicted    1    2    3    4    5  __all__
Actual                                     
1          184    5    1    2    8      200
2            2  195    0    0    3      200
3            4    0  192    1    3      200
4            2    0    0  196    2      200
5            7    3    5    0  185      200
__all__    199  203  198  199  201     1000
precision is: 0.952
recall is: 0.952
F1_score is: 0.952
the performance of BOW_DC:
confusion_matrix is:
Predicted    1    2    3    4    5  __

In [ ]:
##caculate TFIDF based on the cleaned text
import numpy as np
from scipy import sparse
##transform the type of X from list to array then to csr_matrix, so that it can fit into function-'TfidfTransformer'
X_array=np.array(X_set_value)
X_spacy=sparse.csr_matrix(X_array)
from sklearn.feature_extraction.text import TfidfTransformer
transformer=TfidfTransformer()
X_TFIDF=transformer.fit_transform(X_spacy)
X_set_TFIDF=X_TFIDF.toarray()
#print(X_set_TFIDF)
print(len(word_set))
X_train_TFIDF, X_test_TFIDF=X_set_TFIDF[0:round(len(X_set_TFIDF)*0.8)],X_set_value[round(len(X_set_TFIDF)*0.8):len(X_set_TFIDF)]

##built TFIDF_KNN model
import numpy as np
from sklearn import neighbors
TFIDF_KNN=neighbors.KNeighborsClassifier()
Y_pred=cross_val_predict(TFIDF_KNN,X_set_value,Y_set_value,cv=10)
model_performance('TFIDF_KNN', Y_set_value, Y_pred)
TFIDF_KNN_accuracy=cross_val_score(TFIDF_KNN, X_set_value,Y_set_value, cv=10)

##built TFIDF_SVC model
from sklearn.svm import SVC
TFIDF_SVC=SVC(kernel='linear', probability=True)
Y_pred=cross_val_predict(TFIDF_SVC,X_set_value,Y_set_value,cv=10)
model_performance('TFIDF_SVC', Y_set_value, Y_pred)
TFIDF_SVC_accuracy=cross_val_score(TFIDF_SVC, X_set_value,Y_set_value,cv=10)

##built TFIDF_DC model
from sklearn.tree import DecisionTreeClassifier
TFIDF_DC=DecisionTreeClassifier(criterion='entropy')
Y_pred=cross_val_predict(TFIDF_DC,X_set_value,Y_set_value,cv=10)
model_performance('TFIDF_DC', Y_set_value, Y_pred)
TFIDF_DC_accuracy=cross_val_score(TFIDF_DC,X_set_value,Y_set_value,cv=10)

print(f'TFIDF_KNN_accuracy is {TFIDF_KNN_accuracy}')
print(f'TFIDF_SVC_accuracy is {TFIDF_SVC_accuracy}')
print(f'TFIDF_DC_accuracy is {TFIDF_DC_accuracy}')

print('TFIDF_KNN_accuracy: %0.2f (+/- %0.2f)' % (TFIDF_KNN_accuracy.mean(), TFIDF_KNN_accuracy.std()*2))
print('TFIDF_SVC_accuracy: %0.2f (+/- %0.2f)' % (TFIDF_SVC_accuracy.mean(), TFIDF_SVC_accuracy.std()*2))
print('TFIDF_DC_accuracy: %0.2f (+/- %0.2f)' % (TFIDF_DC_accuracy.mean(), TFIDF_DC_accuracy.std()*2))

7754
the performance of TFIDF_KNN:
confusion_matrix is:
Predicted    1    2    3    4    5  __all__
Actual                                     
1          146    2   10    2   40      200
2           14  158   10    0   18      200
3            5    0  185    4    6      200
4            7    0   14  175    4      200
5           32    7   12    5  144      200
__all__    204  167  231  186  212     1000
precision is: 0.808
recall is: 0.808
F1_score is: 0.808
the performance of TFIDF_SVC:
confusion_matrix is:
Predicted    1    2    3    4    5  __all__
Actual                                     
1          184    5    1    2    8      200
2            2  195    0    0    3      200
3            4    0  192    1    3      200
4            2    0    0  196    2      200
5            7    3    5    0  185      200
__all__    199  203  198  199  201     1000
precision is: 0.952
recall is: 0.952
F1_score is: 0.952
the performance of TFIDF_DC:
confusion_matrix is:
Predicted    1    2    3   

In [ ]:
print(f'total running time is {round((time.time()-start_time)/60,2)}')

total running time is 78.62


All the codes below can be ignored. They are record/draft during coding.

In [ ]:
##Ignore-experiment with one book
#find author
import re
text1=gutenberg.raw(textname_list[3])
tokenized_sents=nltk.sent_tokenize(text1)
line1=tokenized_sents[0]
## why not author = re.match(r'[.* by (.*) [0-9]+]',line), the left '[' is not highlighted as the right/close one
#author = (re.match(r'.* by (.*) [0-9]+]',line)).group(1)
#print(author)

In [ ]:
##Ignore-experiment with one book
text1=gutenberg.raw(textname_list[0])
tokenized_sents=nltk.sent_tokenize(text1)
num_count=0
for line in tokenized_sents:
  if num_count<=200:
    if len(line)>=100:
      tokenized_words=nltk.word_tokenize(line)
      label_words = [tokenized_words[0:100],author]
    else:
      break
  num_count+=1

In [ ]:
a='See ,'
b= ' The Wisdom of Father Brown by G. K. Chesterton 1914 I The Absence of Mr Glass THE consulting-rooms of Dr Orion Hood the eminent criminologist and specialist in certain moral disorders lay along the sea-front at Scarborough in a series of very large and well-lighted french windows which showed the North Sea like one endless outer wall of blue-green marble In such a place the sea had something of the monotony of a blue-green dado for the chambers themselves were ruled throughout by a terrible tidiness not unlike the terrible tidiness of the sea It must not be supposed that'
print(len(a))
count_b=0
token_b=nltk.word_tokenize(b)
for tb in token_b:
  if tb not in punctuation:
    count_b+=1
print(count_b)

5
100
